# Analysis of used car listings from eBay Kleinanzeigen

Ebay Kleinanzeigen is a classifieds section of the German eBay website. 

The dataset was originally uploaded to [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data). 
In tis project, a modified version of the full dataset, containing 50,000 data points, will be used instead. The dataset was also "dirtied" to more realistically approximate what would be expected from a scraped dataset. 

The data dictionary provided with the data is seen below:


- `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
-    `name` - Name of the car.
-    `seller` - Whether the seller is private or a dealer.
-    `offerType` - The type of listing
-    `price` - The price on the ad to sell the car.
-    `abtest` - Whether the listing is included in an A/B test.
-    `vehicleType` - The vehicle Type.
-    `yearOfRegistration` - The year in which which year the car was first registered.
-    `gearbox` - The transmission type.
-    `powerPS` - The power of the car in PS.
-    `model` - The car model name.
-    `kilometer` - How many kilometers the car has driven.
-    `monthOfRegistration` - The month in which which year the car was first registered.
-    `fuelType` - What type of fuel the car uses.
-    `brand` - The brand of the car.
-    `notRepairedDamage` - If the car has a damage which is not yet repaired.
-    `dateCreated` - The date on which the eBay listing was created.
-    `nrOfPictures` - The number of pictures in the ad.
-    `postalCode` - The postal code for the location of the vehicle.
-    `lastSeenOnline` - When the crawler saw this ad last online.

The goal of this project is to clean the data and afterwards analyze the used car listings to observe any notable trend or pattern.

First, the libaries that will be necessary to use will be imported.

In [19]:
import pandas as pd
import numpy as np

autos = pd.read_csv("autos.csv", encoding = "Latin-1")

In [20]:
autos.info()

autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Our dataset has 20 columns in total. Some columns like `vehicleType` and `notRepairedDamage` have large numbers of null values, but no column has more than 20% null values. Only 5 of the 20 columns have data stored as integers. The rest have data stored as strings, including the date column. 

In [21]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

## Column renaming

A few of the column names above will have their names changed:

- `yearOfRegistration` to `registration_year`
- `monthOfRegistration` to `registration_month`
- `notRepairedDamage` to `unrepaired_damage`
- `dateCreated` to `ad_created`
- `nrOfPictures` to `num_photos`

Additionally, all column names will be altered from their existing camelcase style, to snakecase (so an underscore separates words)

In [22]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']

autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## Data Exploration and Cleaning

In [23]:
autos.describe(include = "all")

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


In [24]:
print(autos["offer_type"].value_counts())
print(autos["seller"].value_counts())

Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64
privat        49999
gewerblich        1
Name: seller, dtype: int64


Both the `offer_type` and `seller` columns mostly contain one value. Both these columns will be dropped.

The `num_photos` column seems to have 0 or null in all its rows. We will look at this closer.

In [25]:
autos["num_photos"].value_counts()

0    50000
Name: num_photos, dtype: int64

Since the `num_photos` column is filled with only `0`, we'll be dropping this column in addition to `offer_type` and `seller`. 

In [26]:
to_be_dropped = ["num_photos", "offer_type", "seller"]
autos.drop(to_be_dropped, axis = 1, inplace = True)

autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 17 columns):
date_crawled          50000 non-null object
name                  50000 non-null object
price                 50000 non-null object
abtest                50000 non-null object
vehicle_type          44905 non-null object
registration_year     50000 non-null int64
gearbox               47320 non-null object
power_ps              50000 non-null int64
model                 47242 non-null object
odometer              50000 non-null object
registration_month    50000 non-null int64
fuel_type             45518 non-null object
brand                 50000 non-null object
unrepaired_damage     40171 non-null object
ad_created            50000 non-null object
postal_code           50000 non-null int64
last_seen             50000 non-null object
dtypes: int64(4), object(13)
memory usage: 6.5+ MB


The `price` and `odometer` columns contain numeric data stored as strings. One of the reasons they are stored as strings is because the unit they're describing, in this case `$` and `km` respectively, are included in the column's values. 

These units will be removed, and the columns will be converted to numeric types. 

In [28]:
autos["price"].unique()

array(['$5,000', '$8,500', '$8,990', ..., '$385', '$22,200', '$16,995'],
      dtype=object)

The array above shows us that characters like `$` and `,` need to be removed from the price columns.

In [29]:
autos["price"] = autos["price"].str.replace("$", "").str.replace(",", "").astype(float)

autos["price"].head()

0    5000.0
1    8500.0
2    8990.0
3    4350.0
4    1350.0
Name: price, dtype: float64

In [30]:
autos["odometer"].unique()

array(['150,000km', '70,000km', '50,000km', '80,000km', '10,000km',
       '30,000km', '125,000km', '90,000km', '20,000km', '60,000km',
       '5,000km', '100,000km', '40,000km'], dtype=object)

From the odometer column, characters like `,` and `km` need to be removed.

In [31]:
autos["odometer"] = autos["odometer"].str.replace("km", "").str.replace(",", "").astype(float)

autos["odometer"].head()

0    150000.0
1    150000.0
2     70000.0
3     70000.0
4    150000.0
Name: odometer, dtype: float64

The `odometer` column will also be renamed to `odometer_km` to include the information about the unit it describes.

In [32]:
odometer_rename = {"odometer":"odometer_km"}

autos.rename(odometer_rename, axis = 1, inplace = True)

autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 17 columns):
date_crawled          50000 non-null object
name                  50000 non-null object
price                 50000 non-null float64
abtest                50000 non-null object
vehicle_type          44905 non-null object
registration_year     50000 non-null int64
gearbox               47320 non-null object
power_ps              50000 non-null int64
model                 47242 non-null object
odometer_km           50000 non-null float64
registration_month    50000 non-null int64
fuel_type             45518 non-null object
brand                 50000 non-null object
unrepaired_damage     40171 non-null object
ad_created            50000 non-null object
postal_code           50000 non-null int64
last_seen             50000 non-null object
dtypes: float64(2), int64(4), object(11)
memory usage: 6.5+ MB


### Exploring Odometer data more closely

We'll analyze the odometer data more closely, this time looking for values that don't look right (which might be identified by being unrealistically high or low).

In [39]:
print(autos["odometer_km"].unique().shape)

autos["odometer_km"].value_counts().sort_index(ascending=False)

(13,)


150000.0    32424
125000.0     5170
100000.0     2169
90000.0      1757
80000.0      1436
70000.0      1230
60000.0      1164
50000.0      1027
40000.0       819
30000.0       789
20000.0       784
10000.0       264
5000.0        967
Name: odometer_km, dtype: int64

One major point of peculiarity to note here is that despite the fact that the dataframe contains 50,000 data points, there are only 13 unique `odometer_km` values. These 13 values are also rounded.

This very clearly indicates that sellers could not specify their precise odometer amount, **but instead had to choose from a pre-set list of options**. 

Of the 50,000 data points, around 80% of them tracked a mileage of 100,000km or higher, and 32,424 of them tracked a mileage of at least 150,000km. Since 150,000km was the highest option sellers had to choose from, **it is very likely that a large number of the 32,424 people who selected this odometer value had in fact travelled distances far greater than 150,000km**, and when forced to select from a pre-set list of odometer values, they simply chose the value that was closest. 

Apart from the rounded values and the distance selection limit of only 150,000km, there isn't much else off about the values in this column that is noticeable. 

### Exploring Price data more closely

This time we'll analyze the `price` data more closely, once again looking for values that don't look right (which might be identified by them being unrealistically high or low).

In [43]:
print(autos["price"].unique().shape)
print(autos["price"].describe())
autos["price"].value_counts()

(2357,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


0.0           1421
500.0          781
1500.0         734
2500.0         643
1200.0         639
1000.0         639
600.0          531
800.0          498
3500.0         498
2000.0         460
999.0          434
750.0          433
900.0          420
650.0          419
850.0          410
700.0          395
4500.0         394
300.0          384
2200.0         382
950.0          379
1100.0         376
1300.0         371
3000.0         365
550.0          356
1800.0         355
5500.0         340
1250.0         335
350.0          335
1600.0         327
1999.0         322
              ... 
2225.0           1
69997.0          1
139997.0         1
69999.0          1
4780.0           1
8930.0           1
21599.0          1
15911.0          1
10000000.0       1
5180.0           1
919.0            1
1247.0           1
5998.0           1
27020.0          1
21888.0          1
46500.0          1
2001.0           1
2459.0           1
345000.0         1
34940.0          1
2785.0           1
5248.0      

In the price column, there are 2357 unique values. Some of the most common of these are rounded values like "500.0" or "1500.0", but that might just be the inclination of sellers to round off their car price to a certain value.

1,421 cars are listed with a price of $0. These rows might be worth removing.

In [48]:
autos["price"].value_counts().sort_index(ascending = False).head(15)

99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
Name: price, dtype: int64

In [49]:
autos["price"].value_counts().sort_index(ascending = False).tail(15)

18.0       1
17.0       3
15.0       2
14.0       1
13.0       2
12.0       3
11.0       2
10.0       7
9.0        1
8.0        1
5.0        2
3.0        1
2.0        3
1.0      156
0.0     1421
Name: price, dtype: int64

There are several cars with listings below \$20, including 1,421 cars listed at \$0. Since eBay is an auction site, it is entirely possible that cars were legitimately listed with a starting bid of only \$1. 

Also, there are some cars of unrealistically high prices. The most expensive car is listed at almost \$100,000,000, and there are a total of 8 cars at or above the price of \$10,000,000. 

We'll include cars listed at 1, but remove cars above \$350,000 in price, since it appears like the increase in price is gradual until \$350,000, but after that it jumps to extremely unrealistic figures. 

In [53]:
autos = autos[autos["price"].between(1,350001)]
autos["price"].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

After removing the outlier rows that had prices outside of the 1 - 350,001 range, the standard deviation of the price column plummeted from 481,100 to now only 9059. The mean price also fell about 40% - from \$9,840 to now only \$5,889. 

We've also trimmed the number of rows in the `autos` dataframe from 50,000 entries to now only 48,565.

### Exploring the Date columns

5 of the columns in the `autos` dataframe represent date values. Some came from the website itself, and some from the crawler. 

- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

Right now, the `date_crawled`, `last_seen`, and `ad_created` columns are stored as strings. We need to convert the data they store into numeric representation so we can understand it quantitatively. 

The `registration_month` and `registration_year` columns are already represented numerically so we can use methods like `Series.describe()` to understand their distributions without any extra data processing necessary.

First, let's look at how the values in the three string columns are currently formatted:

In [55]:
autos[["date_crawled", "ad_created", "last_seen"]].head(10)

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50
5,2016-03-21 13:47:45,2016-03-21 00:00:00,2016-04-06 09:45:21
6,2016-03-20 17:55:21,2016-03-20 00:00:00,2016-03-23 02:48:59
7,2016-03-16 18:55:19,2016-03-16 00:00:00,2016-04-07 03:17:32
8,2016-03-22 16:51:34,2016-03-22 00:00:00,2016-03-26 18:18:10
9,2016-03-16 13:47:02,2016-03-16 00:00:00,2016-04-06 10:46:35


One thing that appears to be consistent across all three columns is that the first 10 characters, in the form: "YYYY-MM-DD" represent the date. We can thus isolate the date value by splicing the strings stored in the columns to extract these first 10 characters.

#### `date_crawled` column

In [62]:
autos["date_crawled"].str[:10].value_counts(dropna = False, normalize = True).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

In [63]:
autos["date_crawled"].str[:10].value_counts(dropna = False, normalize = True).sort_values()

2016-04-07    0.001400
2016-04-06    0.003171
2016-03-18    0.012911
2016-04-05    0.013096
2016-03-06    0.014043
2016-03-13    0.015670
2016-03-05    0.025327
2016-03-24    0.029342
2016-03-16    0.029610
2016-03-27    0.031092
2016-03-25    0.031607
2016-03-17    0.031628
2016-03-31    0.031834
2016-03-10    0.032184
2016-03-26    0.032204
2016-03-23    0.032225
2016-03-11    0.032575
2016-03-22    0.032987
2016-03-09    0.033090
2016-03-08    0.033296
2016-03-30    0.033687
2016-04-01    0.033687
2016-03-29    0.034099
2016-03-15    0.034284
2016-03-19    0.034778
2016-03-28    0.034860
2016-04-02    0.035478
2016-03-07    0.036014
2016-04-04    0.036487
2016-03-14    0.036549
2016-03-12    0.036920
2016-03-21    0.037373
2016-03-20    0.037887
2016-04-03    0.038608
Name: date_crawled, dtype: float64

The website appears to have been crawled over a period spanning approximately a month, from early March to early April.

For the most part, the distribution of listings on each of the days that the crawling occurred looks uniform.

#### `last_seen` column

In [64]:
(autos["last_seen"].str[:10]
    .value_counts(dropna = False, normalize = True)
    .sort_index()
    )

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

The `last_seen` values above are from the last time the crawler observed any listing. This tells us what day a listing was removed (possibly because a car was sold).

We also see that while the distribution of `last seen` days from 12th March to 4th April is quite uniform, on the last 3 days, from 5th April to 7th April, there is a dramatic spike in the number of listings removed. These values are about 5-10x higher than any of the previous values, which makes it extremely unlikely that these spikes were due to sales spikes during these 3 days. Instead, it's much more likely that these 3 days are simply when the crawling period ended, so they shouldn't be taken to be indicative of car sales.

#### `ad_created` column

In [65]:
(autos["ad_created"]
        .str[:10]
        .value_counts(dropna= False, normalize = True)
        .sort_index())

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

In contrast to the previous 2 columns, there are a very large variety of dates seen here in the `ad_created` column. While a majority of the dates are within about 1-2 months of the ad listing, some dates here reach as far back as June or August of 2015, or about 8-9 months back.

#### `registration_year` column

In [66]:
autos["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

For the most part, the registration year of the car can probably serve as an indicator of its age. Looking at the values in the column, however, we note that we have some extremely odd values. 

The maximum year listed is 9999, which is clearly a false date. The minimum year listed is 1000, which is clearly incorrect as that was well before cars were even invented. The registration_year column is thus clearly in need of some cleaning.  

### Cleaning incorrect `registration_year` data

A car obviously cannot be registered after being listed. As such any date registration_year above 2016 is certainly inaccurate.

It's more tricky to decide what the minimum year should be. It is feasible that it might be as early as the first few decades of the 20th century. 

We'll start by seeing what percentage of lists fall outside of this 1900-2016 range. 

In [73]:
percentage = (autos["registration_year"].between(1900,2016).sum()/autos.shape[0]
 *100)

print("{:.2f}% of cars fall within the 1900-2016 registration years".format(percentage))

96.12% of cars fall within the 1900-2016 registration years


Since an overwhelming majority of our data points within the 1900-2016 registration year range, by dropping the rows that don't belong to that range, we lose less than 4% of our data points. As such, we'll decide to drop them.

In [86]:
autos = autos[autos["registration_year"].between(1900,2016)]

print(autos["registration_year"]
              .value_counts(normalize = True)
              .sort_index(ascending=False).head(20)
     )

print(autos["registration_year"]
              .value_counts(normalize = True)
              .sort_index(ascending=False).head(20)
              .sum()
     )

2016    0.026135
2015    0.008397
2014    0.014203
2013    0.017202
2012    0.028063
2011    0.034768
2010    0.034040
2009    0.044665
2008    0.047450
2007    0.048778
2006    0.057197
2005    0.062895
2004    0.057904
2003    0.057818
2002    0.053255
2001    0.056468
2000    0.067608
1999    0.062060
1998    0.050620
1997    0.041794
Name: registration_year, dtype: float64
0.8713180951564876


We now note that in the modified data-set, a very large portion of cars - approximately 87.1% of them - were first registered in the last 20 years before 2016.

## Exploring Price by Brand

In [93]:
autos["brand"].value_counts(normalize = True)

volkswagen        0.211264
bmw               0.110045
opel              0.107581
mercedes_benz     0.096463
audi              0.086566
ford              0.069900
renault           0.047150
peugeot           0.029841
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008762
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

One of the most obvious things that jumps out from the above value_counts table is the sheer popularity of Volkswagen. It represents 21.1% of cars for sale on its own, which is about as much as the next 2 brands put together. 

The most popular car brands are European (particularly German).

Since brands like Nissan and Jaguar don't have a significant percentage of listing, the scope of this exploration will be limited to brands that comprise at least 2% of cars listed for sale.

In [96]:
brand_amount = autos["brand"].value_counts(normalize=True)
popular_brands = brand_amount[brand_amount > 0.02]
popular_brands.index

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat'],
      dtype='object')

### Determining average Price by Brand

We'll now determine the average price that cars of each brand are listed for.

In [107]:
average_prices = {}

for each in popular_brands.index:
    brand = autos[autos["brand"] == each]
    price_avg = brand["price"].mean()
    average_prices[each] = float(price_avg)
    
average_prices

{'audi': 9336.687453600594,
 'bmw': 8332.820517811953,
 'fiat': 2813.748538011696,
 'ford': 3749.4695065890287,
 'mercedes_benz': 8628.450366422385,
 'opel': 2975.2419354838707,
 'peugeot': 3094.0172290021537,
 'renault': 2474.8646069968195,
 'volkswagen': 5402.410261610221}

Cars from the Audi, BMW, and Mercedes Benz brands are the most expensive by a margin, having average prices in the range of \$8,300-9,300. In contrast, cars belonging to the Renault, Peugeot, Fiat, and Opel brands are much cheaper, with average prices in the \$2,000-3,000 range.

The exceptions to this are the Ford brand, with an average listed price of about \$3,750, and the Volkswagen brand, with its average price of approximately \$5,400. 

What's interesting to note is that the price of Volkswagen cars seems right in between the price range of the more expensive tier of cars, and the less expensive ones. **In some ways, this might be able to explain some of the popularity of Volkswagen cars. It might just be that Volkswagen is widely perceived to be a "best of both worlds" option of sorts, featuring some of the prestige and luxury elements of more costly brands, and yet not being too unaffordable. **

## Exploring Price by Mileage

Now, we'll proceed to use aggregation to understand the average mileage for cars of each brand, whether that has any clear link with mean price.

We'll use the results in the `average_prices` dictionary above to construct a series, and then a dataframe.

In [108]:
average_prices_series = pd.Series(average_prices)

average_prices_series

audi             9336.687454
bmw              8332.820518
fiat             2813.748538
ford             3749.469507
mercedes_benz    8628.450366
opel             2975.241935
peugeot          3094.017229
renault          2474.864607
volkswagen       5402.410262
dtype: float64

We can then create a single-column dataframe from this series object.

In [109]:
brand_mileage_comparison = pd.DataFrame(average_prices_series, columns = ["mean_price"])

brand_mileage_comparison

,mean_price
audi,9336.687454
bmw,8332.820518
fiat,2813.748538
ford,3749.469507
mercedes_benz,8628.450366
opel,2975.241935
peugeot,3094.017229
renault,2474.864607
volkswagen,5402.410262


In [112]:
average_mileage = {}

for each in popular_brands.index:
    brand = autos[autos["brand"] == each]
    avg_mileage = brand["odometer_km"].mean()
    average_mileage[each] = float(avg_mileage)
    
average_mileage

{'audi': 129157.38678544914,
 'bmw': 132572.51313996495,
 'fiat': 117121.9715956558,
 'ford': 124266.01287159056,
 'mercedes_benz': 130788.36331334666,
 'opel': 129310.0358422939,
 'peugeot': 127153.62526920316,
 'renault': 128071.33121308497,
 'volkswagen': 128707.15879132022}

In [114]:
average_mileage_series = pd.Series(average_mileage)

average_mileage_series

audi             129157.386785
bmw              132572.513140
fiat             117121.971596
ford             124266.012872
mercedes_benz    130788.363313
opel             129310.035842
peugeot          127153.625269
renault          128071.331213
volkswagen       128707.158791
dtype: float64

In [115]:
average_mileage_df = pd.DataFrame(average_mileage_series, columns = ["mean_mileage"] )

average_mileage_df

,mean_mileage
audi,129157.386785
bmw,132572.513140
fiat,117121.971596
ford,124266.012872
mercedes_benz,130788.363313
opel,129310.035842
peugeot,127153.625269
renault,128071.331213
volkswagen,128707.158791


Next, we combine the two tables above, and sort the mean_price column in descending order.

In [119]:
brand_mileage_comparison["mean_mileage"] = average_mileage_df

brand_mileage_comparison.sort_values("mean_price", ascending = False)

,mean_price,mean_mileage
audi,9336.687454,129157.386785
mercedes_benz,8628.450366,130788.363313
bmw,8332.820518,132572.513140
volkswagen,5402.410262,128707.158791
ford,3749.469507,124266.012872
peugeot,3094.017229,127153.625269
opel,2975.241935,129310.035842
fiat,2813.748538,117121.971596
renault,2474.864607,128071.331213


The mileage does not vary by brand as much as the mean price does. For the most part, the mileage across each brand looks very uniform, with Fiat cars having average mileages of about 6-7% lower than cars of other brands. 

## Summary

In this project, we cleaned a data-set that had been deliberately "dirtied", and thereafter used the cleaned data-set to derive information about the most popular brands of cars, as well as the mean prices of cars by brand. 

We observed that cars made by European Automobile makers were most popular, representing at least 60% of the total car listings.

In particular, we also noted that the most popular brand of car, Volkswagen, had average listing prices that were in the middle of the "higher-tier" and "lower-tier" cars. We deduced that a large number of people might be drawn to the Volkswagen brand due to perceiving it as a "best of both worlds" option. It possibly satisfies people's latent desire for a luxury brand of car, while also being much more affordable compared to some of the other options like BMW and Audi.

We also gathered that across most brands listed, the average mileage of cars is very uniform, and has no clear correlation to variables like brand and their corresponding average prices.